# BANDGAP_1v_v01
A bandgap in the SKY130 process.  Outputs a 1v reference voltage.

## Table of Contents
- [Introduction and Setup](#introduction)
- [DC Operating Point Test](#dcop_test)
- [Setup Appendix](#setup-appendix)

## Setup
A complete analog and digital open-source IC design environment can be ran inside a Docker 
devcontainer based on the [iic-osic-tools](https://github.com/iic-jku/iic-osic-tools) image from JKU University.  Then  the repo can be opened in vscode directly using the remote devcontainer extension.
This will turn your vscode session into a full analog design environemnt with 
no additional setup required. It will include all the simulators, schematic editors, layout editors, and 
software tools necessary for both analog and digital IC design.

For more details on the environemnt setup see the [Setup Appendix](#setup-appendix) cell.

## Notebook Setup
Initializes some imports and paths that will be needed by the rest of the notebooks.

<a id='introduction'></a>

In [4]:
! source ~/.bashrc
import os
from pathlib import Path
from rich.jupyter import print
from xschem_testbench import xschem_testbench

DEBUG = True
BANDGAP_REPO_ROOT = Path("/workspaces/prjs/bandgapReferenceCircuit")
TESTS_DIR_PATH = BANDGAP_REPO_ROOT / "tests"
SCHEMATIC_DIR_PATH = BANDGAP_REPO_ROOT / "bandgap_sky130_v1"

import ipytest
ipytest.autoconfig()

if DEBUG:
    print(os.environ["PATH"].split(":"))

[
    '/usr/bin',
    '/home/thomas/.local/bin',
    '/home/linuxbrew/.linuxbrew/bin',
    '/home/linuxbrew/.linuxbrew/sbin',
    '/home/thomas/.local/bin',
    '/opt/xyce/xyce_serial/bin',
    '/home/thomas/.nvm/versions/node/v19.2.0/bin',
    '/usr/local/cuda-12.0/bin',
    '/home/thomas/.cargo/bin',
    '/home/linuxbrew/.linuxbrew/bin',
    '/home/linuxbrew/.linuxbrew/sbin',
    '/home/thomas/.local/bin',
    '/usr/local/sbin',
    '/usr/local/bin',
    '/usr/sbin',
    '/usr/bin',
    '/sbin',
    '/bin',
    '/usr/games',
    '/usr/local/games',
    '/snap/bin',
    '/snap/bin',
    '/usr/local/go/bin',
    '/usr/local/go/bin',
    '/foss/tools/xschem/bin',
    '/foss/tools/ngspice/ngspice/bin',
    '/home/linuxbrew/.linuxbrew/bin',
    '/home/linuxbrew/.linuxbrew/sbin',
    '/home/thomas/.local/bin',
    '/opt/xyce/xyce_serial/bin',
    '/home/thomas/.nvm/versions/node/v19.2.0/bin',
    '/usr/local/cuda-12.0/bin',
    '/home/thomas/.cargo/bin',
    '/home/linuxbrew/.linuxbrew/bin',
    '/home/linuxbrew/.linuxbrew/sbin',
    '/home/thomas/.local/bin',
    '/usr/local/sbin',
    '/usr/local/bin',
    '/usr/sbin',
    '/usr/bin',
    '/sbin',
    '/bin',
    '/usr/games',
    '/usr/local/games',
    '/snap/bin',
    '/snap/bin',
    '/usr/local/go/bin',
    '/usr/local/go/bin'
]

# DC Operating Point Test
<a id='dcop_test'></a>
The DC operating point test checks the DC output voltage of the bandgap.

It also checks the terminal voltages (Vds, Vgs, Vgd) of each transistor are 
within the safe operating area. (SOA)  This helps ensure they operate 
reliably over time.

## Run DC Op Simulation

In [5]:

test_name = "dc_op"
test_bench_schematic = "bandgap_1v_v01_dcop_testbench.sch"

dcop_result = xschem_testbench.run(
    name=test_name, 
    schematic_path=SCHEMATIC_DIR_PATH / test_bench_schematic, 
    test_path=TESTS_DIR_PATH / test_name,
    )



Netlisting schematic:
  FROM: bandgap_sky130_v1/bandgap_1v_v01_dcop_testbench.sch
  TO: tests/dc_op/netlist/bandgap_1v_v01_dcop_testbench.spice




CalledProcessError: Command '['xschem', '-q', '-n', '-o', 'tests/dc_op/netlist', '--svg', '--plotfile', 'tests/dc_op/netlist/dc_op.svg', '-l', 'tests/dc_op/netlist/.netlisting.log', '--rcfile', '/foss/pdks/sky130A/libs.tech/xschem/xschemrc', 'bandgap_sky130_v1/bandgap_1v_v01_dcop_testbench.sch']' returned non-zero exit status 1.

## Check DC Op Results

In [ ]:
%%ipytest -qq

%load tests/dc_op/test_op_point.py

## DC Op Performance
Reports the DC operating point performance of the bandgap

# Transient Simulation

This simulation is a time based simulation; the supply, VDD, is ramped up over 5 microseconds. a power on reset pulse is then applied. The circuit then settles into a desired operating point. This simulation performs the test at three temperatures: 0, 27, 70 degrees celcius.

In [ ]:
from spyci import spyci as s

tran1 = s.load_raw("tests/tran/simulation/tsmc_bandgap_real_27degc_vbg.raw")
tran2 = s.load_raw("tests/tran/simulation/tsmc_bandgap_real_0degc_vbg.raw")
tran3 = s.load_raw("tests/tran/simulation/tsmc_bandgap_real_70degc_vbg.raw")

vbg0 = tran2["v(vbg)"][0]

# Setup Appendix
<a id='setup_appendix'></a>